In [5]:
import pymongo as pm

In [6]:
client = pm.MongoClient("mongodb://192.168.0.109:27017")
db = client["tarea"]
collection = db.get_collection("stocks")

In [7]:
# En docs_muerte voy a tener una lista de documentos con el dia y la cantidad total de decesos 
# en el mundo en ese dia.
fase1 = { "$project": { "date_new": { "$dateToString": { "format": "%Y-%m-%d", "date": "$date" } }, 
                        "new_deaths": 1 } }
fase2 = { "$group": { "_id": "$date_new", "total_muertes": { "$sum": "$new_deaths" } } }
docs_muerte = list(db.get_collection("covid19").aggregate( [ fase1, fase2 ] ))

In [ ]:
collection_new = db.create_collection("covid-stocks")

In [12]:
# Recorro docs_muerte, y voy obteniendo de stocks para cada dia en la lista, la 5 companias que han 
# tenido el valor de la accion mas alta. Todo lo guardo en la nueva coleccion.
for doc in docs_muerte:
    fecha = doc['_id']
    fase10 = { "$match": { "DateAsString": fecha } }
    fase11 = { "$sort": { "High": -1 } }
    fase12 = { "$limit" : 5 }
    docs_comp = list(db.get_collection("stocks").aggregate( [ fase10, fase11, fase12 ] ))
    
    # armo un array con la lista de las 5 companias, sera vacio si ese dia no hubo cotizaciones.
    array_stocks = []
    for doc_comp in docs_comp:
        # como no quiero mostrar el simbolo, sino el nombre de la compania
        # voy a buscarlo a la coleccion stock-reference
        nom = list( db.get_collection("stock-reference").find( { "Symbol": doc_comp["Symbol"] } ) )
        nombre_stock = ""
        if ( len(nom) > 0 ):
            nombre_stock = nom[0]["Name"]
        
        array_stocks.append( { "Symbol": doc_comp["Symbol"], "Name": nombre_stock, 
                               "High": doc_comp["High"] } )
    
    # armo el documento y luego lo agrego a la nueva coleccion.
    nuevo_doc = { "DateString": fecha, "total_muertes": doc['total_muertes'], 
                  "Stocks": array_stocks }
    db.get_collection("covid-stocks").insert_one( nuevo_doc )
    

In [28]:
# armo el top 10 de empresas que mas caras tuvieron las acciones en los 1000 dias de mas decesos
cant_dias = 1000
fase20 = { "$sort": { "total_muertes": -1 } }
fase21 = { "$limit" : cant_dias }
fase22 = { "$unwind": "$Stocks" }
fase23 = { "$project": { "DateString": 1, "total_muertes": 1, "Symbol": "$Stocks.Symbol", 
                         "Name": "$Stocks.Name" } }
fase24 = { "$group": { "_id": "$Symbol", "Name" : { "$first": "$Name" }, "total_apariciones": 
                       { "$sum": 1 } } }
fase25 = { "$sort": { "total_apariciones": -1 } }

docs = list(db.get_collection("covid-stocks").aggregate( [ fase20, fase21, fase22, fase23, fase24, 
                                                           fase25 ] ))
docs

[{'_id': 'ASML',
  'Name': 'ASML Holding N.V. New York Registry Shares',
  'total_apariciones': 655},
 {'_id': 'LRCX',
  'Name': 'Lam Research Corporation Common Stock',
  'total_apariciones': 619},
 {'_id': 'AVGO',
  'Name': 'Broadcom Inc. Common Stock',
  'total_apariciones': 602},
 {'_id': 'ADBE', 'Name': 'Adobe Inc. Common Stock', 'total_apariciones': 517},
 {'_id': 'INTU', 'Name': 'Intuit Inc. Common Stock', 'total_apariciones': 477},
 {'_id': 'MPWR',
  'Name': 'Monolithic Power Systems Inc. Common Stock',
  'total_apariciones': 162},
 {'_id': 'ZM',
  'Name': 'Zoom Video Communications Inc. Class A Common Stock',
  'total_apariciones': 99},
 {'_id': 'VRTX', 'Name': '', 'total_apariciones': 45},
 {'_id': 'ANSS', 'Name': 'ANSYS Inc. Common Stock', 'total_apariciones': 42},
 {'_id': 'MDB',
  'Name': 'MongoDB Inc. Class A Common Stock',
  'total_apariciones': 29},
 {'_id': 'KLAC',
  'Name': 'KLA Corporation Common Stock',
  'total_apariciones': 28}]